# Convolutional Variational Autoencoder for dna barcode sequencing embedding 50 dim and LR & RF classifier



## Setup

In [6]:
!pip install tensorflow-probability

# to generate gifs
!pip install imageio
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-f7rx9ye9
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-f7rx9ye9


In [10]:
from IPython import display

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_probability as tfp
import time
import pandas as pd
from numpy import argmax
from numpy import array
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import io
from google.colab import files
import re

In [12]:
from google.colab import files
uploaded = files.upload()
train_features_df = pd.read_csv('train_features.csv')
train_labels_df = pd.read_csv('train_labels.csv')
test_features_df = pd.read_csv('test_features.csv')

## Load the DNA barcode data


In [13]:
def clean_seq(seq_df):
    sequences = seq_df['dna'].str.replace('[^ACGT]','N')
    for i in range(len(sequences)):
        sequences[i] = sequences[i].ljust(1058,'N')
    #print(sequences[0])
    
    sequences_array = np.array(sequences)
    #print(pd.DataFrame(sequences_array))
    seq_list_padded = []
    for i in range(len(sequences_array)):
        list_seq = list(sequences_array[i])
        del list_seq[720:] #truncating sequences after length 720.
        seq_list_padded.append(list_seq)
    print(len(list_seq))
    replace_map = {'A': 1, 'C': 2, 'G': 3, 'T': 4, 'N': 5}
    integer_encoded = []
    for i in range(len(seq_list_padded)):
        C = (pd.Series(seq_list_padded[i])).map(replace_map) #convert the list to a pandas series temporarily before mapping
        integer_encoded.append(list(C))
    data = np.array([1, 2, 3, 4, 5])
    data = data.reshape(-1, 1) 
    integer_encoded = np.array(integer_encoded).astype(int)
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoder.fit(data)
    one_hot_encoded_seqs = []
    for i in range(len(integer_encoded)):
        integer_encoded_tmp = integer_encoded[i].reshape(len(integer_encoded[i]), 1)
        onehot_encoded = np.array(onehot_encoder.transform(integer_encoded_tmp)).flatten()
        one_hot_encoded_seqs.append(onehot_encoded)
    one_hot_encoded_seqs_array = np.array(one_hot_encoded_seqs)
    one_hot_encoded_seqs_array = one_hot_encoded_seqs_array.reshape(len(one_hot_encoded_seqs_array), 60, 60) #reshaping to (60,60)
    one_hot_encoded_seqs_array = np.expand_dims(one_hot_encoded_seqs_array, axis=3)
    
    return one_hot_encoded_seqs_array

def load_n_encode_labels(df):
    labels = df['labels']
    softmax_layer = len(set(labels))
    le = LabelEncoder()
    le.fit(labels)
    label_seq = le.transform(labels)
    
    return label_seq, le, softmax_layer

def decode_labels(encoded_predict_labels, le):
    test_predictions = le.inverse_transform(encoded_predict_labels)
    
    return test_predictions

In [14]:
one_hot_encoded_seqs_array_train = clean_seq(train_features_df)
one_hot_encoded_seqs_array_test = clean_seq(test_features_df)
one_hot_encoded_seqs_array = np.row_stack((one_hot_encoded_seqs_array_train, one_hot_encoded_seqs_array_test))
label_seq, le, softmax_layer = load_n_encode_labels(train_labels_df)
X_train = one_hot_encoded_seqs_array
X_validation = one_hot_encoded_seqs_array_test
#X_train, X_validation, y_train, y_validation = train_test_split(one_hot_encoded_seqs_array, label_seq, test_size=0.2)
train_size = len(X_train)
batch_size = 32
validation_size = len(X_validation)

720
720


In [15]:
print(one_hot_encoded_seqs_array_train.shape)
print(one_hot_encoded_seqs_array_test.shape)

(12906, 60, 60, 1)
(8306, 60, 60, 1)


In [16]:
print(X_train.shape)
#print(y_train.shape)
print(X_validation.shape)
#print(y_validation.shape)

(21212, 60, 60, 1)
(8306, 60, 60, 1)


In [17]:
X_train = X_train.astype(np.float32)
X_validation = X_validation.astype(np.float32)
print(X_train.dtype)
print(X_validation.dtype)

float32
float32


## Use *tf.data* to batch and shuffle the data

In [18]:
train_dataset = (tf.data.Dataset.from_tensor_slices(X_train)
                 .shuffle(train_size).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(X_validation)
                .shuffle(validation_size).batch(batch_size))

## Define CVAE model

In [19]:
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(60, 60, 1)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=15*15*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(15, 15, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

In [20]:
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

## Training

In [21]:
epochs = 50
# set the dimensionality of the latent space to a plane for visualization later
latent_dim = 50

model = CVAE(latent_dim)

print(model.encoder.summary())
print(model.decoder.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               1254500   
Total params: 1,273,316
Trainable params: 1,273,316
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 7200)              367200    
___________________

In [22]:
tf.config.run_functions_eagerly(True)

In [23]:
for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in train_dataset:
    train_step(model, train_x, optimizer)
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in test_dataset:
    loss(compute_loss(model, test_x))
  elbo = -loss.result()
  display.clear_output(wait=False)
  print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
        .format(epoch, elbo, end_time - start_time))

Epoch: 50, Test set ELBO: -234.6387176513672, time elapse for current epoch: 11.392330169677734


In [24]:
## Generate embedding for Train data

In [25]:
mean, logvar = model.encode(one_hot_encoded_seqs_array_train)
train_embedding = model.reparameterize(mean, logvar)

print(train_embedding)
print(train_embedding.shape)

tf.Tensor(
[[-1.5802376  -1.2517327  -0.14466335 ... -2.1828485   0.10308895
  -1.1642257 ]
 [-0.20291764 -2.0560195   0.6987632  ...  0.56530476  0.15143682
  -1.067295  ]
 [-0.4041378   1.4878821   0.7742104  ... -1.2107522  -2.5717123
   0.93492836]
 ...
 [-0.8557013  -1.7069868   0.9909979  ...  0.6357273  -0.22385727
  -0.16221741]
 [ 1.101889   -1.1398277  -0.38348067 ... -0.23872104 -0.42603981
  -0.572437  ]
 [ 1.2138689  -0.14824048  0.03710124 ... -2.2271652   1.1860719
  -0.7092244 ]], shape=(12906, 50), dtype=float32)
(12906, 50)


In [26]:
## Generate embedding for Test data

In [27]:
mean, logvar = model.encode(one_hot_encoded_seqs_array_test)
test_embedding = model.reparameterize(mean, logvar)

print(test_embedding)
print(test_embedding.shape)

tf.Tensor(
[[ 7.5766727e-02 -6.8397105e-01 -5.2378243e-01 ...  1.0389309e+00
  -5.1163405e-02  1.5363646e-01]
 [ 1.2755394e-04  2.6131229e+00  1.9072046e+00 ... -3.2568377e-01
   1.0774726e+00  1.5425061e+00]
 [-1.6793849e+00 -3.2717049e-01  5.1898026e-01 ... -1.0220902e+00
  -1.1570249e+00 -1.4720592e-01]
 ...
 [-5.7026392e-01  1.6811582e-01 -1.5922544e+00 ...  7.9567623e-01
   9.2103785e-01  3.6093354e-01]
 [ 8.6318439e-01  5.1874185e-01  1.7914237e-01 ... -8.8333404e-01
   3.9404798e-01 -1.3009837e+00]
 [-6.3685483e-01 -5.7832688e-02 -7.5279599e-01 ...  9.9138230e-02
  -3.1684837e-01 -1.0302316e+00]], shape=(8306, 50), dtype=float32)
(8306, 50)


In [28]:
## DNA sequence classification using Random Forest

In [30]:
train_embedding = np.array(train_embedding)
print(train_embedding)

[[-1.5802376  -1.2517327  -0.14466335 ... -2.1828485   0.10308895
  -1.1642257 ]
 [-0.20291764 -2.0560195   0.6987632  ...  0.56530476  0.15143682
  -1.067295  ]
 [-0.4041378   1.4878821   0.7742104  ... -1.2107522  -2.5717123
   0.93492836]
 ...
 [-0.8557013  -1.7069868   0.9909979  ...  0.6357273  -0.22385727
  -0.16221741]
 [ 1.101889   -1.1398277  -0.38348067 ... -0.23872104 -0.42603981
  -0.572437  ]
 [ 1.2138689  -0.14824048  0.03710124 ... -2.2271652   1.1860719
  -0.7092244 ]]


In [32]:
import numpy as np
label_seq = np.ravel(label_seq)
x_sample_train, x_test, y_sample_train, y_test = train_test_split(train_embedding, label_seq, test_size=0.2) #train and test split

In [33]:
# evaluate random forest algorithm for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier

# define the model
model_RF = RandomForestClassifier()
model_RF.fit(x_sample_train, y_sample_train)

# evaluate the model
score = model_RF.score(x_test, y_test)
# report performance
print('Accuracy: %.3f' % (score))

Accuracy: 0.919


In [ ]:
test_embedding_array = np.array(test_embedding)
encoded_predict_labels = clf.predict(test_embedding_array)

test_predictions = decode_labels(encoded_predict_labels, le)

print(test_predictions)

[1162    8  625 ...  625  625  510]


In [ ]:
test_embedding_array = np.array(test_embedding)
encoded_predict_labels = clf.predict(test_embedding_array)
print(encoded_predict_labels)
test_predictions = decode_labels(encoded_predict_labels, le)
print(test_predictions)

[1150    7  617 ...  617  617  503]
[1162    8  625 ...  625  625  510]


In [ ]:
#test_data_prediction and submission file creation

In [ ]:
def get_seq_ids(filename_df):
    ids = filename_df['id']
    
    return np.array(ids)

test_ids = get_seq_ids(test_features_df)
print(test_ids)

[   1    2    3 ... 8304 8305 8306]


In [ ]:
frames = [pd.DataFrame(test_ids), pd.DataFrame(test_predictions)]
output_data= np.concatenate(frames, axis=1)
output_df = pd.DataFrame(output_data)
output_df.to_csv('dna_barcode_seq_submission2_CVAE_embedding_RF_Classifier.csv', index=False,  header=["id","labels"])

In [ ]:
from google.colab import files
files.download("dna_barcode_seq_submission2_CVAE_embedding_RF_Classifier.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>